In [1]:
import os
os.getcwd()

'/Users/happyeon/Desktop/data_science/play_data'

### GS25에서 CSRF 값 가져오기

In [2]:
import requests
from bs4 import BeautifulSoup as BS
url = "http://gs25.gsretail.com/gscvs/ko/store-services/locations#;"
r = requests.get(url)
bs = BS(r.text)
csrf = bs.find("form", id="CSRFForm").find("input")['value']


In [3]:
csrf

'71402d96-934c-43b4-acbf-ab5cdf5675b7'

### Session 유지 기법

In [4]:
with requests.Session() as s:
    r = s.get(url)
    bs = BS(r.text)
    csrf = bs.find("form", id="CSRFForm").find("input")['value']


In [5]:
csrf

'684363fa-0d96-4629-a5e2-1ee2fe45eca7'

### 전국 GS 편의점 모든 점포

In [12]:
payload = {"pageNum"  : "1",
"pageSize"  : "100",
"searchShopName"  : "",
"searchSido"  : "11",
"searchGugun"  : "",
"searchDong"  : "",
"searchType"  : "",
"searchTypeService"  : "0",
"searchTypeToto"  : "0",
"searchTypeCafe25"  : "0",
"searchTypeInstant"  : "0",
"searchTypeDrug"  : "0",
"searchTypeSelf25"  : "0",
"searchTypePost"  : "0",
"searchTypeATM"  : "0",
"searchTypeWithdrawal"  : "0",
"searchTypeTaxrefund"  : "0",
"searchTypeSmartAtm"  : "0",
"searchTypeSelfCookingUtensils"  : "0",
"searchTypeDeliveryService"  : "0",
}


In [15]:

master = {x.text : x['value'] for x in bs.find("select", id="stb1").findAll("option")[1:]}

In [19]:
import json
import pandas as pd
post_url = "http://gs25.gsretail.com/gscvs/ko/store-services/locationList?CSRFToken={}"
total = []
with requests.Session() as s:
    r = s.get(url)
    bs = BS(r.text)
    csrf = bs.find("form", id="CSRFForm").find("input")['value']
    for code in master.values():
        payload['pageSize'] = 5000
        payload['searchSido'] = code
        r2 = s.post(post_url.format(csrf), data=payload)
        total.append(pd.DataFrame(json.loads(r2.json())['results']))


In [20]:
gs = pd.concat(total)

In [21]:
gs

,offeringService,shopCode,longs,address,shopName,lat
0,[post],VQ037,37.52831065,"서울 양천구 목동서로159-1 (목1동 917-1, CBS방송국1층)",GS25CBS점,126.87486792
1,"[cafe25, post]",VO395,37.5783899,"서울 마포구 매봉산로75, 1층 107호 (상암동 1610)",GS25DDMC점,126.89294205
2,"[cafe25, drug, post]",VW887,37.57410234,"서울 서대문구 가재울미래로2, 203동 101호 (남가좌동 385, 2단지)",GS25DMC가재울점,126.91639401
3,"[cafe25, instant, post]",V9A48,37.50235508,"서울 강남구 테헤란로223, 1층 101호 (역삼동 677-25, 큰길타워빌딩)",GS25DXLAB점,127.04069826
4,"[drug, withdrawal]",VI989,37.55513349,서울 강동구 고덕로39 (암사3동 441-14),GS25G고덕점,127.12956973
...,...,...,...,...,...,...
376,"[cafe25, instant, drug, post]",V9377,33.51836478,제주 제주시 동화로1길49-9 (화북일동 13-6),GS25화북주공점,126.57739082
377,"[cafe25, post]",V4S83,33.52321296,제주 제주시 진남로33 (화북일동 4201-2),GS25화북진남점,126.56390997
378,"[cafe25, instant, withdrawal]",VY067,33.24402459,제주 서귀포시 안덕면 화순중앙로94 (화순리 358-3),GS25화순아일랜드점,126.33601411
379,"[cafe25, drug, withdrawal, self_cook]",VDB29,33.24653847,"제주 서귀포시 안덕면 화순로142, C동 1층 (화순리 1055-3)",GS25화순육거리점,126.33119176


In [22]:
gs['시'] = gs['address'].apply(lambda x : x.split()[0])

In [23]:
gs['시'].value_counts()

경기       4462
서울       3107
경남       1019
부산        905
인천        897
경북        799
충남        757
강원        655
대구        629
충북        527
전북        463
대전        458
전남        434
제주        379
광주        350
울산        331
세종        100
부산광역시       2
제주특별자       2
Name: 시, dtype: int64

In [41]:
gs.loc[gs['offeringService'].notnull(), '의료'] = gs[gs['offeringService'].notnull()]['offeringService'].apply(lambda x :
                                    'Y' if 'drug' in x else 'N')


In [42]:
gs['의료'].value_counts(normalize=True)

Y    0.82795
N    0.17205
Name: 의료, dtype: float64

In [44]:
gs[gs['의료'] == 'Y'].groupby(['시'])['의료'].count()

시
강원        402
경기       3517
경남        845
경북        633
광주        312
대구        576
대전        403
부산        808
부산광역시       2
서울       2594
세종         56
울산        280
인천        746
전남        338
전북        397
제주        221
제주특별자       1
충남        571
충북        421
Name: 의료, dtype: int64

In [45]:
gs['의료'] = gs['의료'].fillna('N')

In [46]:
gs.isnull().sum()

offeringService    426
shopCode             0
longs                0
address              0
shopName             0
lat                  0
시                    0
의료                   0
dtype: int64

In [47]:
gs[gs['offeringService'].notnull()][gs[gs['offeringService'].notnull()]['offeringService'].apply(lambda x :  \
                     len(x) == 11)]

,offeringService,shopCode,longs,address,shopName,lat,시,의료
159,"[toto, cafe25, instant, drug, post, atm, withd...",V9373,33.489818,제주 제주시 도령로76(연동 251-57),GS25연동신광점,126.48660204,제주,Y


### 전국 세븐 일레븐

#### 셀레니움은 requests가 JS를 return할 때 가져다 쓴다.

JS는 데이터를 추출하기에 부적합하기 때문

In [122]:
from selenium import webdriver
driver = webdriver.Chrome()

In [123]:
url = "https://www.7-eleven.co.kr/"
driver.get(url)


In [124]:
from selenium.webdriver.common.by import By
target_store = "#header > div > div > div.head_util > a.util_store.store_open"


In [ ]:
def click(selector):
    driver.find_element(By.CSS_SELECTOR, selector).click()
    return 

In [125]:
click(target_store)

In [78]:
page = driver.page_source

In [79]:
def return_seven(page):
    global seven_total
    seven = BS(page)
    seven_total = []
    for tmp in seven.find("div", class_="list_stroe").findAll("li"):
        seven_dict = {}
        seven_dict['offeringService'] = [x['alt'] for x in tmp.findAll("img")]
        seven_dict['shopName'] = tmp.find("span").text.strip()
        seven_dict['address']= " ".join(tmp.findAll("span")[-2].text.split())    
        if len(seven_dict['address']) < 2 :
            seven_dict['address']= " ".join(tmp.findAll("span")[-3].text.split())    
        _, lat, lon = re.findall("(?<=\().+(?=\))", tmp.find('a')['href'])[0].split(",")
        seven_dict['longs'] = lon
        seven_dict['lat'] = lat
        seven_total.append(seven_dict)
    return pd.DataFrame(seven_total)

1. 웹드라이버 시/구 페이지
2. 시/구 세븐일레븐 점포
3. output: 전국 세븐일레븐 점포

In [126]:
import pandas as pd,time
korea_seven = []
for i in range(2,19):
    click('#storeLaySido')
    time.sleep(2)
    click(f'#storeLaySido > option:nth-child({i})')
    gugun_length = len(BS(driver.page_source).find('select', id='storeLayGu').findAll("option"))
    for j in range(2,gugun_length+1):
        try:
            click("#storeLayGu")
            time.sleep(2)
            click(f"#storeLayGu > option:nth-child({j})")
            click("#storeButton1")
            time.sleep(2)
            korea_seven.append(return_seven(driver.page_source))
        except Exception as ex:
            print(f"e --> {ex}")
        

pd.concat(korea_seven)
        

e --> list index out of range
e --> list index out of range


,offeringService,shopName,address,longs,lat
0,"[의약품, 시디]",강릉5주공점,강원도 강릉시 월대산로 43,128.91519608,37.760113214
1,[],강릉가작로점,강원도 강릉시 가작로274번길 19 1층,128.903952839001,37.7690256763271
2,[],강릉강문스타점,강원도 강릉시 창해로350번길 21 (강문동),128.917594053613,37.7958884030248
3,"[24시간, 카페, 의약품, 페덱스서비스, 스마트픽]",강릉강문점,"강원도 강릉시 해안로 298-7 B블록 101호, 102호(강문동)",128.91970091425162,37.79232227004757
4,[],강릉강변로점,강원도 강릉시 강변로 262,128.897338589574,37.749106410644
...,...,...,...,...,...
63,[],충주충청대로점,충청북도 충주시 주덕읍 충청대로 2323,127.778647057936,36.9635309277246
64,[],충주탄금대로점,충청북도 충주시 탄금대로 224 (칠금동),127.907334358097,36.9843867570316
65,"[24시간, 카페, 치킨, 의약품, 예약주문, 페덱스서비스, 시디, 스마트픽]",충주호암점,충청북도 충주시 호암중앙2로 32 101호(호암동),127.94029493,36.955341151
66,[],충주호암LH점,충청북도 충주시 호암수청2로 56,127.927072494247,36.9554585474234


In [127]:
korea_seven = pd.concat(korea_seven)
korea_seven

,offeringService,shopName,address,longs,lat
0,"[의약품, 시디]",강릉5주공점,강원도 강릉시 월대산로 43,128.91519608,37.760113214
1,[],강릉가작로점,강원도 강릉시 가작로274번길 19 1층,128.903952839001,37.7690256763271
2,[],강릉강문스타점,강원도 강릉시 창해로350번길 21 (강문동),128.917594053613,37.7958884030248
3,"[24시간, 카페, 의약품, 페덱스서비스, 스마트픽]",강릉강문점,"강원도 강릉시 해안로 298-7 B블록 101호, 102호(강문동)",128.91970091425162,37.79232227004757
4,[],강릉강변로점,강원도 강릉시 강변로 262,128.897338589574,37.749106410644
...,...,...,...,...,...
63,[],충주충청대로점,충청북도 충주시 주덕읍 충청대로 2323,127.778647057936,36.9635309277246
64,[],충주탄금대로점,충청북도 충주시 탄금대로 224 (칠금동),127.907334358097,36.9843867570316
65,"[24시간, 카페, 치킨, 의약품, 예약주문, 페덱스서비스, 시디, 스마트픽]",충주호암점,충청북도 충주시 호암중앙2로 32 101호(호암동),127.94029493,36.955341151
66,[],충주호암LH점,충청북도 충주시 호암수청2로 56,127.927072494247,36.9554585474234
